In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSN%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFDSN%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_flux_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
t = 0.0
Nmax = 10000
avg_pc_iterations = 0.0
flog = open('FDSN%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting, study case: %i\n' % k)
flog.close()
flog = open('FDSN%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    # mapping
    tempF.curr[:] = tempS.curr[s2f]
    
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=tempF.curr
    )
    
    while True:
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        fluxF.curr[:] = solverF.get_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        fluxS.curr[:] = fluxF.curr[f2s]
        # update solid flux interface
        solverS.update_flux_interface(interface_tag, fluxS.curr)
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = tempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
flog.close()

step=0, pc_iterations=9.
step=1, pc_iterations=5.
step=2, pc_iterations=5.
step=3, pc_iterations=5.
step=4, pc_iterations=4.
step=5, pc_iterations=4.
step=6, pc_iterations=4.
step=7, pc_iterations=4.
step=8, pc_iterations=4.
step=9, pc_iterations=3.
step=10, pc_iterations=3.
step=11, pc_iterations=3.
step=12, pc_iterations=3.
step=13, pc_iterations=3.
step=14, pc_iterations=4.
step=15, pc_iterations=3.
step=16, pc_iterations=3.
step=17, pc_iterations=3.
step=18, pc_iterations=3.
step=19, pc_iterations=3.
step=20, pc_iterations=3.
step=21, pc_iterations=3.
step=22, pc_iterations=3.
step=23, pc_iterations=3.
step=24, pc_iterations=3.
step=25, pc_iterations=3.
step=26, pc_iterations=3.
step=27, pc_iterations=3.
step=28, pc_iterations=3.
step=29, pc_iterations=3.
step=30, pc_iterations=3.
step=31, pc_iterations=3.
step=32, pc_iterations=3.
step=33, pc_iterations=3.
step=34, pc_iterations=3.
step=35, pc_iterations=3.
step=36, pc_iterations=2.
step=37, pc_iterations=3.
step=38, pc_iterations

step=308, pc_iterations=2.
step=309, pc_iterations=2.
step=310, pc_iterations=2.
step=311, pc_iterations=2.
step=312, pc_iterations=2.
step=313, pc_iterations=2.
step=314, pc_iterations=2.
step=315, pc_iterations=2.
step=316, pc_iterations=2.
step=317, pc_iterations=2.
step=318, pc_iterations=2.
step=319, pc_iterations=2.
step=320, pc_iterations=2.
step=321, pc_iterations=2.
step=322, pc_iterations=2.
step=323, pc_iterations=2.
step=324, pc_iterations=2.
step=325, pc_iterations=2.
step=326, pc_iterations=2.
step=327, pc_iterations=2.
step=328, pc_iterations=2.
step=329, pc_iterations=2.
step=330, pc_iterations=2.
step=331, pc_iterations=2.
step=332, pc_iterations=2.
step=333, pc_iterations=2.
step=334, pc_iterations=2.
step=335, pc_iterations=2.
step=336, pc_iterations=2.
step=337, pc_iterations=2.
step=338, pc_iterations=2.
step=339, pc_iterations=2.
step=340, pc_iterations=2.
step=341, pc_iterations=2.
step=342, pc_iterations=2.
step=343, pc_iterations=2.
step=344, pc_iterations=2.
s

step=612, pc_iterations=2.
step=613, pc_iterations=2.
step=614, pc_iterations=2.
step=615, pc_iterations=2.
step=616, pc_iterations=2.
step=617, pc_iterations=2.
step=618, pc_iterations=2.
step=619, pc_iterations=2.
step=620, pc_iterations=2.
step=621, pc_iterations=2.
step=622, pc_iterations=2.
step=623, pc_iterations=2.
step=624, pc_iterations=2.
step=625, pc_iterations=2.
step=626, pc_iterations=2.
step=627, pc_iterations=2.
step=628, pc_iterations=2.
step=629, pc_iterations=2.
step=630, pc_iterations=2.
step=631, pc_iterations=2.
step=632, pc_iterations=2.
step=633, pc_iterations=2.
step=634, pc_iterations=2.
step=635, pc_iterations=2.
step=636, pc_iterations=2.
step=637, pc_iterations=2.
step=638, pc_iterations=2.
step=639, pc_iterations=2.
step=640, pc_iterations=2.
step=641, pc_iterations=2.
step=642, pc_iterations=2.
step=643, pc_iterations=2.
step=644, pc_iterations=2.
step=645, pc_iterations=2.
step=646, pc_iterations=2.
step=647, pc_iterations=2.
step=648, pc_iterations=2.
s

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFDSN%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.91609245 0.92280578 0.92763899 0.93159074 0.93495102 0.93779915
 0.94033688 0.94260747 0.94468391 0.94663812 0.94842788 0.95011422
 0.95168927 0.95318402 0.9545886  0.95592599 0.95719158 0.95840507
 0.9595597  0.96065896 0.96171215 0.96271776 0.96368551 0.96461017
 0.9654959  0.96634503 0.96716478 0.96795147 0.96871043 0.96943951
 0.97013953 0.97081524 0.97146617 0.97209148 0.97269894 0.97328227
 0.97384469 0.97438658 0.97491234 0.97541927 0.97590987 0.97638409
 0.97684133 0.97728337 0.97771081 0.9781243  0.97852391 0.97891141
 0.97928534 0.97964767 0.97999659 0.98033611 0.98066386 0.98097984
 0.98128627 0.98158076 0.98186659 0.98214138 0.9824066  0.98266308
 0.98290955 0.98314731 0.98337467 0.9835948  0.98380507 0.98400579
 0.9841989  0.9843812  0.98455569 0.98472053 0.98487569 0.98502206
 0.98515822 0.98528562 0.98540199 0.98550993 0.98560637 0.98569203
 0.98576758 0.98582977 0.98588124 0.98591884 0.98594267 0.98595291
 0.98594792 0.98592613 0.98588662 0.98582903 0.9857493  0.9856